In [1]:
import pandas as pd
import numpy as np

salesdata_df = pd.read_csv('/Users/kadam/Desktop/CS689/Project/Data/processedsalesdata.csv')
print(salesdata_df.head())

   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  MX-2014-143658  2014-10-02  2014-10-06  Standard Class    SC-20575   
1       2  MX-2012-155047  2012-10-15  2012-10-20  Standard Class    KW-16570   
2       3  MX-2012-155047  2012-10-15  2012-10-20  Standard Class    KW-16570   
3       4  MX-2012-155047  2012-10-15  2012-10-20  Standard Class    KW-16570   
4       5  MX-2012-155047  2012-10-15  2012-10-20  Standard Class    KW-16570   

    Customer Name   Segment           City             State  ...  \
0    Sonia Cooley  Consumer    Mexico City  Distrito Federal  ...   
1  Kelly Williams  Consumer  Dos Quebradas         Risaralda  ...   
2  Kelly Williams  Consumer  Dos Quebradas         Risaralda  ...   
3  Kelly Williams  Consumer  Dos Quebradas         Risaralda  ...   
4  Kelly Williams  Consumer  Dos Quebradas         Risaralda  ...   

  Sub-Category                        Product Name   Sales Quantity Discount  \
0       Labels  Ho

In [2]:
incomedata_df = pd.read_csv('/Users/kadam/Desktop/CS689/Project/Data/processedincomedata.csv')
print(incomedata_df.head())

       Country  ISO  Region_ID  Year  GDP_PPC_USD2011  Population  \
0  Afghanistan  afg         27  1967           1268.0  27722282.0   
1  Afghanistan  afg         27  1967           1268.0  27722282.0   
2  Afghanistan  afg         27  1967           1268.0  27722282.0   
3  Afghanistan  afg         27  1967           1268.0  27722282.0   
4  Afghanistan  afg         27  1967           1268.0  27722282.0   

  Income_Category  Net_Income  Gini_coeff  
0              d1    0.032262    0.290652  
1              d2    0.050450    0.290652  
2              d3    0.061768    0.290652  
3              d4    0.071800    0.290652  
4              d5    0.081803    0.290652  


### Product Dimension

In [117]:
Product_df = salesdata_df[['Product ID','Product Name', 'Category', 'Sub-Category']]

Product_df.head()

,Product ID,Product Name,Category,Sub-Category
0,OFF-LA-10002782,"Hon File Folder Labels, Adjustable",Office Supplies,Labels
1,FUR-FU-10004015,"Tenex Clock, Durable",Furniture,Furnishings
2,FUR-BO-10002352,"Ikea 3-Shelf Cabinet, Mobile",Furniture,Bookcases
3,OFF-BI-10004428,"Cardinal Binder, Clear",Office Supplies,Binders
4,OFF-AR-10004594,"Sanford Canvas, Water Color",Office Supplies,Art


In [118]:
duplicate_counts = Product_df.groupby(['Product ID','Product Name', 'Category', 'Sub-Category']).size().reset_index(name='Count')
duplicates = duplicate_counts[duplicate_counts['Count'] > 1] ##To find duplicate values
print(duplicates)

             Product ID                       Product Name    Category  \
0      FUR-ADV-10000002     Advantus Photo Frame, Duo Pack   Furniture   
1      FUR-ADV-10000108          Advantus Clock, Erganomic   Furniture   
2      FUR-ADV-10000183        Advantus Photo Frame, Black   Furniture   
3      FUR-ADV-10000188  Advantus Stacking Tray, Erganomic   Furniture   
5      FUR-ADV-10000571          Advantus Frame, Erganomic   Furniture   
...                 ...                                ...         ...   
10762  TEC-STA-10003925     StarTech Card Printer, Durable  Technology   
10763  TEC-STA-10004181           StarTech Inkjet, Durable  Technology   
10764  TEC-STA-10004536          StarTech Inkjet, Wireless  Technology   
10765  TEC-STA-10004542       StarTech Calculator, Durable  Technology   
10766  TEC-STA-10004834      StarTech Receipt Printer, Red  Technology   

      Sub-Category  Count  
0      Furnishings      2  
1      Furnishings      3  
2      Furnishings      8  

In [119]:
## dropping the duplicate values
unique_Product_df = Product_df.drop_duplicates(subset=['Product ID','Product Name', 'Category', 'Sub-Category'])
#unique_Product_df.head()
display(unique_Product_df)

,Product ID,Product Name,Category,Sub-Category
0,OFF-LA-10002782,"Hon File Folder Labels, Adjustable",Office Supplies,Labels
1,FUR-FU-10004015,"Tenex Clock, Durable",Furniture,Furnishings
2,FUR-BO-10002352,"Ikea 3-Shelf Cabinet, Mobile",Furniture,Bookcases
3,OFF-BI-10004428,"Cardinal Binder, Clear",Office Supplies,Binders
4,OFF-AR-10004594,"Sanford Canvas, Water Color",Office Supplies,Art
...,...,...,...,...
50913,FUR-DAN-10001557,"Dania Floating Shelf Set, Pine",Furniture,Bookcases
50924,OFF-KRA-10002441,"Kraft Business Envelopes, Recycled",Office Supplies,Envelopes
51075,OFF-GRE-10003213,"Green Bar Memo Slips, Premium",Office Supplies,Paper
51155,FUR-CHR-10000888,"Chromcraft Computer Table, Fully Assembled",Furniture,Tables


In [120]:
# Making the dimension into SCD2
from datetime import datetime

def apply_scd2(unique_Product_df):
    unique_Product_df['Effective_From'] = pd.to_datetime(datetime.now().date())  # Setting the current date as the effective date for new records
    unique_Product_df['Effective_To'] = pd.to_datetime('2262-04-11')  # Set a future date for the current records

    unique_Product_df['CurrentFlag'] = Product_df.duplicated(subset=['Product ID'], keep='last')

    # Update the existing records
    unique_Product_df.loc[unique_Product_df['CurrentFlag'], 'Effective_To'] = pd.to_datetime(datetime.now().date())

    return unique_Product_df

# Applying SCD2
Product_df_scd2 = apply_scd2(unique_Product_df)

print(Product_df_scd2)

             Product ID                                Product Name  \
0       OFF-LA-10002782          Hon File Folder Labels, Adjustable   
1       FUR-FU-10004015                        Tenex Clock, Durable   
2       FUR-BO-10002352                Ikea 3-Shelf Cabinet, Mobile   
3       OFF-BI-10004428                      Cardinal Binder, Clear   
4       OFF-AR-10004594                 Sanford Canvas, Water Color   
...                 ...                                         ...   
50913  FUR-DAN-10001557              Dania Floating Shelf Set, Pine   
50924  OFF-KRA-10002441          Kraft Business Envelopes, Recycled   
51075  OFF-GRE-10003213               Green Bar Memo Slips, Premium   
51155  FUR-CHR-10000888  Chromcraft Computer Table, Fully Assembled   
51185  OFF-OIC-10004622                  OIC Thumb Tacks, Bulk Pack   

              Category Sub-Category Effective_From Effective_To  CurrentFlag  
0      Office Supplies       Labels     2023-12-10   2023-12-10     

/var/folders/6n/g0nqd9qx36g6tzn8__zgtbp80000gn/T/ipykernel_39797/4031985727.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_Product_df['Effective_From'] = pd.to_datetime(datetime.now().date())  # Setting the current date as the effective date for new records
/var/folders/6n/g0nqd9qx36g6tzn8__zgtbp80000gn/T/ipykernel_39797/4031985727.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_Product_df['Effective_To'] = pd.to_datetime('2262-04-11')  # Set a future date for the current records
/var

In [121]:
ProductDim = Product_df_scd2.rename(columns={
    'Product ID': 'Product_id',
    'Product Name': 'Product_name',
    'Category': 'Product_category',
    'Sub-Category': 'Product_subcategory'
})

print(ProductDim)

             Product_id                                Product_name  \
0       OFF-LA-10002782          Hon File Folder Labels, Adjustable   
1       FUR-FU-10004015                        Tenex Clock, Durable   
2       FUR-BO-10002352                Ikea 3-Shelf Cabinet, Mobile   
3       OFF-BI-10004428                      Cardinal Binder, Clear   
4       OFF-AR-10004594                 Sanford Canvas, Water Color   
...                 ...                                         ...   
50913  FUR-DAN-10001557              Dania Floating Shelf Set, Pine   
50924  OFF-KRA-10002441          Kraft Business Envelopes, Recycled   
51075  OFF-GRE-10003213               Green Bar Memo Slips, Premium   
51155  FUR-CHR-10000888  Chromcraft Computer Table, Fully Assembled   
51185  OFF-OIC-10004622                  OIC Thumb Tacks, Bulk Pack   

      Product_category Product_subcategory Effective_From Effective_To  \
0      Office Supplies              Labels     2023-12-10   2023-12-10   

In [122]:
ProductDim = ProductDim.copy()

ProductDim.reset_index(drop=False, inplace=True)
ProductDim.rename(columns={'index': 'ProdDim_Id'}, inplace=True)
ProductDim['ProdDim_Id'] += 1  # Start the index at 1
ProductDim.head()

,ProdDim_Id,Product_id,Product_name,Product_category,Product_subcategory,Effective_From,Effective_To,CurrentFlag
0,1,OFF-LA-10002782,"Hon File Folder Labels, Adjustable",Office Supplies,Labels,2023-12-10,2023-12-10,True
1,2,FUR-FU-10004015,"Tenex Clock, Durable",Furniture,Furnishings,2023-12-10,2023-12-10,True
2,3,FUR-BO-10002352,"Ikea 3-Shelf Cabinet, Mobile",Furniture,Bookcases,2023-12-10,2023-12-10,True
3,4,OFF-BI-10004428,"Cardinal Binder, Clear",Office Supplies,Binders,2023-12-10,2023-12-10,True
4,5,OFF-AR-10004594,"Sanford Canvas, Water Color",Office Supplies,Art,2023-12-10,2023-12-10,True


In [123]:
ProductDim.to_csv('/Users/kadam/Desktop/CS689/Project/Data/ProductDim.csv', index=False)  

print(f'DataFrame has been successfully written')

DataFrame has been successfully written


In [115]:
duplicate_counts = ProductDim.groupby(['Product_id','Product_name', 'Product_category', 'Product_subcategory']).size().reset_index(name='Count')
duplicates = duplicate_counts[duplicate_counts['Count'] > 1] ##To find duplicate values
print(duplicates)

Empty DataFrame
Columns: [Product_id, Product_name, Product_category, Product_subcategory, Count]
Index: []


### Shipment Dimension

In [9]:
Shipment_df = salesdata_df[['Ship Date', 'Ship Mode']]
Shipment_df.head()

,Ship Date,Ship Mode
0,2014-10-06,Standard Class
1,2012-10-20,Standard Class
2,2012-10-20,Standard Class
3,2012-10-20,Standard Class
4,2012-10-20,Standard Class


In [10]:
Shipment_df = Shipment_df.copy()

Shipment_df.reset_index(drop=False, inplace=True)
Shipment_df.rename(columns={'index': 'Id'}, inplace=True)
Shipment_df['Id'] += 1  # Start the index at 1
display(Shipment_df)


,Id,Ship Date,Ship Mode
0,1,2014-10-06,Standard Class
1,2,2012-10-20,Standard Class
2,3,2012-10-20,Standard Class
3,4,2012-10-20,Standard Class
4,5,2012-10-20,Standard Class
...,...,...,...
51285,51286,2012-09-07,First Class
51286,51287,2012-09-07,First Class
51287,51288,2012-09-07,First Class
51288,51289,2012-09-07,First Class


In [11]:
# Function to apply SCD2
def apply_scd2(Shipment_df):
    Shipment_df['Effective_From'] = pd.to_datetime(datetime.now().date())  # Setting the current date as the effective date for new records
    Shipment_df['Effective_To'] = pd.to_datetime('2262-04-11')  # Set a distant future date for the current records

    # Identify the records to be updated
    Shipment_df['CurrentFlag'] = Shipment_df.duplicated(subset=['Ship Mode'], keep='last')

    # Update the existing records
    Shipment_df.loc[Shipment_df['CurrentFlag'], 'Effective_To'] = pd.to_datetime(datetime.now().date())

    return Shipment_df

# Applying SCD2
df_shipments_scd2 = apply_scd2(Shipment_df.copy())  # Use .copy() to avoid modifying the original DataFrame

# Display the resulting DataFrame
print(df_shipments_scd2)

          Id   Ship Date       Ship Mode Effective_From Effective_To  \
0          1  2014-10-06  Standard Class     2023-12-10   2023-12-10   
1          2  2012-10-20  Standard Class     2023-12-10   2023-12-10   
2          3  2012-10-20  Standard Class     2023-12-10   2023-12-10   
3          4  2012-10-20  Standard Class     2023-12-10   2023-12-10   
4          5  2012-10-20  Standard Class     2023-12-10   2023-12-10   
...      ...         ...             ...            ...          ...   
51285  51286  2012-09-07     First Class     2023-12-10   2023-12-10   
51286  51287  2012-09-07     First Class     2023-12-10   2023-12-10   
51287  51288  2012-09-07     First Class     2023-12-10   2023-12-10   
51288  51289  2012-09-07     First Class     2023-12-10   2262-04-11   
51289  51290  2011-08-29  Standard Class     2023-12-10   2262-04-11   

       CurrentFlag  
0             True  
1             True  
2             True  
3             True  
4             True  
...      

In [12]:
ShipmentDim = df_shipments_scd2.rename(columns={
    'Id': 'shipment_id',
    'Ship Date': 'shipment_date',
    'Ship Mode': 'shipment_mode'
})

print(ShipmentDim)

       shipment_id shipment_date   shipment_mode Effective_From Effective_To  \
0                1    2014-10-06  Standard Class     2023-12-10   2023-12-10   
1                2    2012-10-20  Standard Class     2023-12-10   2023-12-10   
2                3    2012-10-20  Standard Class     2023-12-10   2023-12-10   
3                4    2012-10-20  Standard Class     2023-12-10   2023-12-10   
4                5    2012-10-20  Standard Class     2023-12-10   2023-12-10   
...            ...           ...             ...            ...          ...   
51285        51286    2012-09-07     First Class     2023-12-10   2023-12-10   
51286        51287    2012-09-07     First Class     2023-12-10   2023-12-10   
51287        51288    2012-09-07     First Class     2023-12-10   2023-12-10   
51288        51289    2012-09-07     First Class     2023-12-10   2262-04-11   
51289        51290    2011-08-29  Standard Class     2023-12-10   2262-04-11   

       CurrentFlag  
0             True

In [13]:
ShipmentDim.to_csv('/Users/kadam/Desktop/CS689/Project/Data/ShipmentDim.csv', index=False)  

print(f'DataFrame has been successfully written')

DataFrame has been successfully written


### Order Dimension

In [129]:
order_df = salesdata_df[['Order ID', 'Order Date', 'Order Priority']]
order_df.head()

,Order ID,Order Date,Order Priority
0,MX-2014-143658,2014-10-02,Medium
1,MX-2012-155047,2012-10-15,Medium
2,MX-2012-155047,2012-10-15,Medium
3,MX-2012-155047,2012-10-15,Medium
4,MX-2012-155047,2012-10-15,Medium


In [130]:
OrderDim = order_df.rename(columns={
    'Order ID': 'order_id',
    'Order Date': 'order_date',
    'Order Priority': 'order_priority'
})

print(OrderDim)

             order_id  order_date order_priority
0      MX-2014-143658  2014-10-02         Medium
1      MX-2012-155047  2012-10-15         Medium
2      MX-2012-155047  2012-10-15         Medium
3      MX-2012-155047  2012-10-15         Medium
4      MX-2012-155047  2012-10-15         Medium
...               ...         ...            ...
51285    HU-2012-7730  2012-09-05           High
51286    HU-2012-7730  2012-09-05           High
51287    HU-2012-7730  2012-09-05           High
51288    HU-2012-7730  2012-09-05           High
51289    IZ-2011-5750  2011-08-25         Medium

[51290 rows x 3 columns]


In [131]:
duplicate_counts2 = OrderDim.groupby(['order_id','order_date', 'order_priority']).size().reset_index(name='Count')
duplicates2 = duplicate_counts2[duplicate_counts2['Count'] > 1] ##To find duplicate values
print(duplicates2)

           order_id  order_date order_priority  Count
0      AE-2011-9160  2011-10-03         Medium      2
1      AE-2013-1130  2013-10-14           High      2
2      AE-2013-1530  2013-12-31           High      2
4      AE-2014-3830  2014-12-13         Medium      6
7      AG-2011-1390  2011-08-16         Medium      2
...             ...         ...            ...    ...
25744  ZI-2014-5970  2014-06-10           High      3
25746  ZI-2014-6740  2014-11-30           High      6
25748  ZI-2014-7160  2014-10-30         Medium      3
25749  ZI-2014-7610  2014-03-24         Medium      2
25751  ZI-2014-9550  2014-12-18         Medium      2

[12790 rows x 4 columns]


In [132]:
## dropping the duplicate values
OrderDim = OrderDim.drop_duplicates(subset=['order_id','order_date', 'order_priority'])
display(OrderDim)

,order_id,order_date,order_priority
0,MX-2014-143658,2014-10-02,Medium
1,MX-2012-155047,2012-10-15,Medium
6,MX-2013-134096,2013-09-27,Medium
11,MX-2013-156335,2013-03-05,Medium
13,MX-2014-121923,2014-09-24,Low
...,...,...,...
51276,TU-2013-7800,2013-08-09,Medium
51280,IV-2012-7600,2012-09-21,Medium
51281,RS-2014-4620,2014-08-14,Medium
51284,HU-2012-7730,2012-09-05,High


In [133]:
OrderDim = OrderDim.copy()

OrderDim.reset_index(drop=False, inplace=True)
OrderDim.rename(columns={'index': 'OrderId'}, inplace=True)
OrderDim['OrderId'] += 1  # Start the index at 1
OrderDim.head()

,OrderId,order_id,order_date,order_priority
0,1,MX-2014-143658,2014-10-02,Medium
1,2,MX-2012-155047,2012-10-15,Medium
2,7,MX-2013-134096,2013-09-27,Medium
3,12,MX-2013-156335,2013-03-05,Medium
4,14,MX-2014-121923,2014-09-24,Low


In [134]:
OrderDim.to_csv('/Users/kadam/Desktop/CS689/Project/Data/OrderDim.csv', index=False)  

print(f'DataFrame has been successfully written')

DataFrame has been successfully written


### Market Dimension

In [19]:
market_df = salesdata_df[['Segment', 'Market', 'Region']]
market_df.head()

,Segment,Market,Region
0,Consumer,LATAM,North
1,Consumer,LATAM,South
2,Consumer,LATAM,South
3,Consumer,LATAM,South
4,Consumer,LATAM,South


In [20]:
duplicate_counts = market_df.groupby(['Segment','Market', 'Region']).size().reset_index(name='Count')
duplicates = duplicate_counts[duplicate_counts['Count'] > 1] ##To find duplicate values
print(duplicates)

        Segment  Market          Region  Count
0      Consumer    APAC    Central Asia   1042
1      Consumer    APAC      North Asia   1170
2      Consumer    APAC         Oceania   1837
3      Consumer    APAC  Southeast Asia   1650
4      Consumer  Africa          Africa   2381
5      Consumer  Canada          Canada    202
6      Consumer    EMEA            EMEA   2538
7      Consumer      EU         Central   3057
8      Consumer      EU           North   1090
9      Consumer      EU           South   1039
10     Consumer   LATAM       Caribbean    828
11     Consumer   LATAM         Central   1513
12     Consumer   LATAM           North   1378
13     Consumer   LATAM           South   1602
14     Consumer      US         Central   1212
15     Consumer      US            East   1469
16     Consumer      US           South    838
17     Consumer      US            West   1672
18    Corporate    APAC    Central Asia    613
19    Corporate    APAC      North Asia    708
20    Corpora

In [21]:
## dropping the duplicate values
unique_market_df = market_df.drop_duplicates(subset=['Segment', 'Market', 'Region'])
unique_market_df.head()


,Segment,Market,Region
0,Consumer,LATAM,North
1,Consumer,LATAM,South
11,Consumer,LATAM,Central
13,Home Office,LATAM,Central
16,Corporate,LATAM,Central


In [22]:
unique_market_df = unique_market_df.copy()

unique_market_df.reset_index(drop=False, inplace=True)
unique_market_df.rename(columns={'index': 'Market_Id'}, inplace=True)
unique_market_df['Market_Id'] += 1  # Start the index at 1
unique_market_df.head()

,Market_Id,Segment,Market,Region
0,1,Consumer,LATAM,North
1,2,Consumer,LATAM,South
2,12,Consumer,LATAM,Central
3,14,Home Office,LATAM,Central
4,17,Corporate,LATAM,Central


In [23]:
MarketDim = unique_market_df.rename(columns={
    'Segment': 'market_segment',
    'Market': 'market_category',
    'Region': 'market_region'
})

print(MarketDim.head())

   Market_Id market_segment market_category market_region
0          1       Consumer           LATAM         North
1          2       Consumer           LATAM         South
2         12       Consumer           LATAM       Central
3         14    Home Office           LATAM       Central
4         17      Corporate           LATAM       Central


In [24]:
MarketDim.to_csv('/Users/kadam/Desktop/CS689/Project/Data/MarketDim.csv', index=False)  

print(f'DataFrame has been successfully written')

DataFrame has been successfully written


### Customer Dimension

In [138]:
customer_df = salesdata_df[['Customer ID', 'Customer Name', 'City', 'State', 'Country']]
customer_df.head()

,Customer ID,Customer Name,City,State,Country
0,SC-20575,Sonia Cooley,Mexico City,Distrito Federal,Mexico
1,KW-16570,Kelly Williams,Dos Quebradas,Risaralda,Colombia
2,KW-16570,Kelly Williams,Dos Quebradas,Risaralda,Colombia
3,KW-16570,Kelly Williams,Dos Quebradas,Risaralda,Colombia
4,KW-16570,Kelly Williams,Dos Quebradas,Risaralda,Colombia


In [139]:
duplicate_counts3 = customer_df.groupby(['Customer ID', 'Customer Name', 'City', 'State', 'Country']).size().reset_index(name='Count')
duplicates3 = duplicate_counts3[duplicate_counts3['Count'] > 1] ##To find duplicate values
print(duplicates3)

      Customer ID       Customer Name           City             State  \
1        AA-10315          Alex Avila   Bacolod City   Western Visayas   
5        AA-10315          Alex Avila       Garforth           England   
8        AA-10315          Alex Avila       Marikina  National Capital   
9        AA-10315          Alex Avila    Mexico City  Distrito Federal   
10       AA-10315          Alex Avila    Minneapolis         Minnesota   
...           ...                 ...            ...               ...   
24949    ZD-21925  Zuschuss Donatelli         Manila  National Capital   
24951    ZD-21925  Zuschuss Donatelli          Milan          Lombardy   
24952    ZD-21925  Zuschuss Donatelli       Richmond           Indiana   
24954    ZD-21925  Zuschuss Donatelli          Rouen    Upper Normandy   
24955    ZD-21925  Zuschuss Donatelli  San Francisco        California   

              Country  Count  
1         Philippines      2  
5      United Kingdom      8  
8         Philippi

In [140]:
## dropping the duplicate values
unique_customer_df = customer_df.drop_duplicates(subset=['Customer ID', 'Customer Name', 'City', 'State', 'Country'])
display(unique_customer_df)

,Customer ID,Customer Name,City,State,Country
0,SC-20575,Sonia Cooley,Mexico City,Distrito Federal,Mexico
1,KW-16570,Kelly Williams,Dos Quebradas,Risaralda,Colombia
6,DP-13000,Darren Powers,São Paulo,São Paulo,Brazil
11,TB-21250,Tim Brockman,Managua,Managua,Nicaragua
13,PK-18910,Paul Knutson,Managua,Managua,Nicaragua
...,...,...,...,...,...
51276,MY-8295,Muhammed Yedwab,Adana,Adana,Turkey
51280,TC-11145,Theresa Coyne,Bouake,Vallee Du Bandama,Cote d'Ivoire
51281,CR-2625,Corey Roper,Astrakhan',Astrakhan',Russia
51284,NF-8385,Natalie Fritzler,Budapest,Budapest,Hungary


In [141]:
#  Function to apply SCD2
def apply_scd2(unique_customer_df, keys):
    unique_customer_df['Effective_From'] = pd.to_datetime(datetime.now().date())  # Setting the current date as the effective date for new records
    unique_customer_df['Effective_To'] = pd.to_datetime('2262-04-11')  # Set a future date for the current records

    unique_customer_df['CurrentFlag'] = unique_customer_df.duplicated(subset=keys, keep='last')

    # Update the existing records
    unique_customer_df.loc[unique_customer_df['CurrentFlag'], 'Effective_To'] = pd.to_datetime(datetime.now().date())

    return unique_customer_df

# Applying SCD2
df_customers_scd2 = apply_scd2(unique_customer_df.copy(), keys=['Customer ID', 'City', 'State'])

# Display the resulting DataFrame
print(df_customers_scd2)


      Customer ID     Customer Name           City              State  \
0        SC-20575      Sonia Cooley    Mexico City   Distrito Federal   
1        KW-16570    Kelly Williams  Dos Quebradas          Risaralda   
6        DP-13000     Darren Powers      São Paulo          São Paulo   
11       TB-21250      Tim Brockman        Managua            Managua   
13       PK-18910      Paul Knutson        Managua            Managua   
...           ...               ...            ...                ...   
51276     MY-8295   Muhammed Yedwab          Adana              Adana   
51280    TC-11145     Theresa Coyne         Bouake  Vallee Du Bandama   
51281     CR-2625       Corey Roper     Astrakhan'         Astrakhan'   
51284     NF-8385  Natalie Fritzler       Budapest           Budapest   
51289    SZ-10035        Sam Zeldin        Baghdad            Baghdad   

             Country Effective_From Effective_To  CurrentFlag  
0             Mexico     2023-12-10   2262-04-11        Fal

In [142]:
CustomerDim = df_customers_scd2.rename(columns={
    'Customer ID': 'customer_id',
    'Customer Name': 'customer_name',
    'City': 'customer_city',
    'State': 'customer_state',
    'Country': 'customer_country'
})

print(CustomerDim)


      customer_id     customer_name  customer_city     customer_state  \
0        SC-20575      Sonia Cooley    Mexico City   Distrito Federal   
1        KW-16570    Kelly Williams  Dos Quebradas          Risaralda   
6        DP-13000     Darren Powers      São Paulo          São Paulo   
11       TB-21250      Tim Brockman        Managua            Managua   
13       PK-18910      Paul Knutson        Managua            Managua   
...           ...               ...            ...                ...   
51276     MY-8295   Muhammed Yedwab          Adana              Adana   
51280    TC-11145     Theresa Coyne         Bouake  Vallee Du Bandama   
51281     CR-2625       Corey Roper     Astrakhan'         Astrakhan'   
51284     NF-8385  Natalie Fritzler       Budapest           Budapest   
51289    SZ-10035        Sam Zeldin        Baghdad            Baghdad   

      customer_country Effective_From Effective_To  CurrentFlag  
0               Mexico     2023-12-10   2262-04-11       

In [143]:
CustomerDim = CustomerDim.copy()

CustomerDim.reset_index(drop=False, inplace=True)
CustomerDim.rename(columns={'index': 'customerId'}, inplace=True)
CustomerDim['customerId'] += 1  # Start the index at 1
CustomerDim.head()

,customerId,customer_id,customer_name,customer_city,customer_state,customer_country,Effective_From,Effective_To,CurrentFlag
0,1,SC-20575,Sonia Cooley,Mexico City,Distrito Federal,Mexico,2023-12-10,2262-04-11,False
1,2,KW-16570,Kelly Williams,Dos Quebradas,Risaralda,Colombia,2023-12-10,2262-04-11,False
2,7,DP-13000,Darren Powers,São Paulo,São Paulo,Brazil,2023-12-10,2262-04-11,False
3,12,TB-21250,Tim Brockman,Managua,Managua,Nicaragua,2023-12-10,2262-04-11,False
4,14,PK-18910,Paul Knutson,Managua,Managua,Nicaragua,2023-12-10,2262-04-11,False


In [144]:
CustomerDim.to_csv('/Users/kadam/Desktop/CS689/Project/Data/CustomerDim.csv', index=False)  

print(f'DataFrame has been successfully written')

DataFrame has been successfully written


### Location Dimension

In [31]:
location_df = salesdata_df[['Region', 'City', 'State', 'Country']]
location_df.head()

,Region,City,State,Country
0,North,Mexico City,Distrito Federal,Mexico
1,South,Dos Quebradas,Risaralda,Colombia
2,South,Dos Quebradas,Risaralda,Colombia
3,South,Dos Quebradas,Risaralda,Colombia
4,South,Dos Quebradas,Risaralda,Colombia


In [32]:
duplicate_counts = location_df.groupby(['Region','City', 'State', 'Country']).size().reset_index(name='Count')
duplicates = duplicate_counts[duplicate_counts['Count'] > 1] ##To find duplicate values
print(duplicates)

      Region         City          State        Country  Count
0     Africa          Aba           Abia        Nigeria     25
1     Africa    Abakaliki         Ebonyi        Nigeria      2
2     Africa     Abeokuta           Ogun        Nigeria      8
3     Africa      Abidjan        Lagunes  Cote d'Ivoire     76
4     Africa    Abu Kabir  Ash Sharqiyah          Egypt      4
...      ...          ...            ...            ...    ...
3812    West      Visalia     California  United States      4
3813    West  West Jordan           Utah  United States      5
3814    West  Westminster     California  United States     17
3816    West     Woodland     California  United States      3
3818    West         Yuma        Arizona  United States      4

[3303 rows x 5 columns]


In [33]:
## dropping the duplicate values
unique_location_df = location_df.drop_duplicates(subset=['Region','City', 'State', 'Country'])

print(unique_location_df)

        Region             City                  State     Country
0        North      Mexico City       Distrito Federal      Mexico
1        South    Dos Quebradas              Risaralda    Colombia
6        South        São Paulo              São Paulo      Brazil
11     Central          Managua                Managua   Nicaragua
14       South         Brasília       Federal District      Brazil
...        ...              ...                    ...         ...
51104     EMEA        Iranshahr  Sistan Va Baluchestan        Iran
51183     EMEA             Kars                   Kars      Turkey
51195     EMEA           Kalush        Ivano-Frankivsk     Ukraine
51198   Africa  Kafr ash Shaykh        Kafr Ash Shaykh       Egypt
51229     EMEA          Jizzakh                Jizzakh  Uzbekistan

[3819 rows x 4 columns]


In [34]:
unique_location_df = unique_location_df.copy()

unique_location_df.reset_index(drop=False, inplace=True)
unique_location_df.rename(columns={'index': 'Location_Id'}, inplace=True)
unique_location_df['Location_Id'] += 1  # Start the index at 1
display(unique_location_df)

,Location_Id,Region,City,State,Country
0,1,North,Mexico City,Distrito Federal,Mexico
1,2,South,Dos Quebradas,Risaralda,Colombia
2,7,South,São Paulo,São Paulo,Brazil
3,12,Central,Managua,Managua,Nicaragua
4,15,South,Brasília,Federal District,Brazil
...,...,...,...,...,...
3814,51105,EMEA,Iranshahr,Sistan Va Baluchestan,Iran
3815,51184,EMEA,Kars,Kars,Turkey
3816,51196,EMEA,Kalush,Ivano-Frankivsk,Ukraine
3817,51199,Africa,Kafr ash Shaykh,Kafr Ash Shaykh,Egypt


In [35]:
LocationDim = unique_location_df.copy()

In [36]:
LocationDim.head()

,Location_Id,Region,City,State,Country
0,1,North,Mexico City,Distrito Federal,Mexico
1,2,South,Dos Quebradas,Risaralda,Colombia
2,7,South,São Paulo,São Paulo,Brazil
3,12,Central,Managua,Managua,Nicaragua
4,15,South,Brasília,Federal District,Brazil


In [37]:
LocationDim.to_csv('/Users/kadam/Desktop/CS689/Project/Data/LocationDim.csv', index=False)  

print(f'DataFrame has been successfully written')

DataFrame has been successfully written


### Date Dimension

In [38]:
Date_df = salesdata_df[['Order Date', 'Order Year', 'Order Month']]
Date_df.head()

,Order Date,Order Year,Order Month
0,2014-10-02,2014,10
1,2012-10-15,2012,10
2,2012-10-15,2012,10
3,2012-10-15,2012,10
4,2012-10-15,2012,10


In [39]:
Date_df = Date_df.rename(columns={
    'Order Date': 'Date',
    'Order Month': 'Month',
    'Order Year': 'Year'
})

print(Date_df)


             Date  Year  Month
0      2014-10-02  2014     10
1      2012-10-15  2012     10
2      2012-10-15  2012     10
3      2012-10-15  2012     10
4      2012-10-15  2012     10
...           ...   ...    ...
51285  2012-09-05  2012      9
51286  2012-09-05  2012      9
51287  2012-09-05  2012      9
51288  2012-09-05  2012      9
51289  2011-08-25  2011      8

[51290 rows x 3 columns]


In [40]:
duplicate_counts = Date_df.groupby(['Date','Year', 'Month']).size().reset_index(name='Count')
duplicates = duplicate_counts[duplicate_counts['Count'] > 1] ##To find duplicate values
print(duplicates)

            Date  Year  Month  Count
0     2011-01-01  2011      1      6
2     2011-01-03  2011      1     20
3     2011-01-04  2011      1     16
4     2011-01-05  2011      1      7
5     2011-01-06  2011      1      8
...          ...   ...    ...    ...
1425  2014-12-27  2014     12     54
1426  2014-12-28  2014     12     13
1427  2014-12-29  2014     12    116
1428  2014-12-30  2014     12     79
1429  2014-12-31  2014     12     62

[1404 rows x 4 columns]


In [41]:
## dropping the duplicate values
unique_Date_df = Date_df.drop_duplicates(subset=['Date','Year', 'Month'])

print(unique_Date_df)

             Date  Year  Month
0      2014-10-02  2014     10
1      2012-10-15  2012     10
6      2013-09-27  2013      9
11     2013-03-05  2013      3
13     2014-09-24  2014      9
...           ...   ...    ...
47443  2012-02-05  2012      2
48575  2011-04-24  2011      4
49549  2011-01-02  2011      1
49728  2013-01-20  2013      1
49894  2014-02-09  2014      2

[1430 rows x 3 columns]


In [42]:
unique_Date_df = unique_Date_df.copy()

unique_Date_df.reset_index(drop=False, inplace=True)
unique_Date_df.rename(columns={'index': 'DateDim_Id'}, inplace=True)
unique_Date_df['DateDim_Id'] += 1  # Start the index at 1
display(unique_Date_df)

,DateDim_Id,Date,Year,Month
0,1,2014-10-02,2014,10
1,2,2012-10-15,2012,10
2,7,2013-09-27,2013,9
3,12,2013-03-05,2013,3
4,14,2014-09-24,2014,9
...,...,...,...,...
1425,47444,2012-02-05,2012,2
1426,48576,2011-04-24,2011,4
1427,49550,2011-01-02,2011,1
1428,49729,2013-01-20,2013,1


In [43]:

unique_Date_df['Date'] = pd.to_datetime(unique_Date_df['Date'])

unique_Date_df['Day_of_Week'] = unique_Date_df['Date'].dt.day_name()
unique_Date_df['Week_Number'] = unique_Date_df['Date'].dt.week

print(unique_Date_df)

      DateDim_Id       Date  Year  Month Day_of_Week  Week_Number
0              1 2014-10-02  2014     10    Thursday           40
1              2 2012-10-15  2012     10      Monday           42
2              7 2013-09-27  2013      9      Friday           39
3             12 2013-03-05  2013      3     Tuesday           10
4             14 2014-09-24  2014      9   Wednesday           39
...          ...        ...   ...    ...         ...          ...
1425       47444 2012-02-05  2012      2      Sunday            5
1426       48576 2011-04-24  2011      4      Sunday           16
1427       49550 2011-01-02  2011      1      Sunday           52
1428       49729 2013-01-20  2013      1      Sunday            3
1429       49895 2014-02-09  2014      2      Sunday            6

[1430 rows x 6 columns]


/var/folders/6n/g0nqd9qx36g6tzn8__zgtbp80000gn/T/ipykernel_39797/2741724198.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  unique_Date_df['Week_Number'] = unique_Date_df['Date'].dt.week


In [44]:
DateDim = unique_Date_df.copy()

In [45]:
DateDim.head()

,DateDim_Id,Date,Year,Month,Day_of_Week,Week_Number
0,1,2014-10-02,2014,10,Thursday,40
1,2,2012-10-15,2012,10,Monday,42
2,7,2013-09-27,2013,9,Friday,39
3,12,2013-03-05,2013,3,Tuesday,10
4,14,2014-09-24,2014,9,Wednesday,39


In [46]:
DateDim.to_csv('/Users/kadam/Desktop/CS689/Project/Data/DateDim.csv', index=False)  

print(f'DataFrame has been successfully written')

DataFrame has been successfully written


### Year Dimension

In [47]:
Year_df = salesdata_df[['Order Year']]

In [48]:
Year_df = Year_df.rename(columns={
    'Order Year': 'Year'
})

print(Year_df)

       Year
0      2014
1      2012
2      2012
3      2012
4      2012
...     ...
51285  2012
51286  2012
51287  2012
51288  2012
51289  2011

[51290 rows x 1 columns]


In [49]:
duplicate_counts = Year_df.groupby(['Year']).size().reset_index(name='Count')
duplicates = duplicate_counts[duplicate_counts['Count'] > 1] ##To find duplicate values
print(duplicates)

   Year  Count
0  2011   8998
1  2012  10962
2  2013  13799
3  2014  17531


In [50]:
## dropping the duplicate values
unique_Year_df = Year_df.drop_duplicates(subset=['Year'])

print(unique_Year_df)

    Year
0   2014
1   2012
6   2013
48  2011


In [51]:
incomeYear_df = incomedata_df[['Year']]

In [52]:
duplicate_counts = incomeYear_df.groupby(['Year']).size().reset_index(name='Count')
duplicates = duplicate_counts[duplicate_counts['Count'] > 1] ##To find duplicate values
print(duplicates)

    Year  Count
0   1958     10
1   1965     10
2   1967   1730
3   1968   1730
4   1969   1730
5   1970   1740
6   1971   1730
7   1972   1730
8   1973   1740
9   1974   1730
10  1975   1730
11  1976   1740
12  1977   1730
13  1978   1730
14  1979   1730
15  1980   1740
16  1981   1730
17  1982   1730
18  1983   1730
19  1984   1730
20  1985   1740
21  1986   1730
22  1987   1730
23  1988   1740
24  1989   1740
25  1990   1730
26  1991   1740
27  1992   1740
28  1993   1730
29  1994   1740
30  1995   1750
31  1996   1730
32  1997   1750
33  1998   1740
34  1999   1750
35  2000   1750
36  2001   1740
37  2002   1740
38  2003   1730
39  2004   1730
40  2005   1730
41  2006   1760
42  2007   1750
43  2008   1760
44  2009   1750
45  2010   1760
46  2011   1740
47  2012   1740
48  2013   1740
49  2014   1760
50  2015   1760


In [53]:
## dropping the duplicate values
unique_incomeYear_df = incomeYear_df.drop_duplicates(subset=['Year'])

print(unique_incomeYear_df)

       Year
0      1967
10     1968
20     1969
30     1970
40     1971
50     1972
60     1973
70     1974
80     1975
90     1976
100    1977
110    1978
120    1979
130    1980
140    1981
150    1982
160    1983
170    1984
180    1985
190    1986
200    1987
210    1988
220    1989
230    1990
240    1991
250    1992
260    1993
270    1994
280    1995
290    1996
300    1997
310    1998
320    1999
330    2000
340    2001
350    2002
360    2003
370    2004
380    2005
390    2006
400    2007
410    2008
420    2009
430    2010
440    2011
450    2012
460    2013
470    2014
480    2015
17660  1958
17670  1965


In [54]:
filtered_Year_df = unique_incomeYear_df.merge(unique_Year_df, on=['Year'], how='left', indicator=True)

print(filtered_Year_df)


    Year     _merge
0   1967  left_only
1   1968  left_only
2   1969  left_only
3   1970  left_only
4   1971  left_only
5   1972  left_only
6   1973  left_only
7   1974  left_only
8   1975  left_only
9   1976  left_only
10  1977  left_only
11  1978  left_only
12  1979  left_only
13  1980  left_only
14  1981  left_only
15  1982  left_only
16  1983  left_only
17  1984  left_only
18  1985  left_only
19  1986  left_only
20  1987  left_only
21  1988  left_only
22  1989  left_only
23  1990  left_only
24  1991  left_only
25  1992  left_only
26  1993  left_only
27  1994  left_only
28  1995  left_only
29  1996  left_only
30  1997  left_only
31  1998  left_only
32  1999  left_only
33  2000  left_only
34  2001  left_only
35  2002  left_only
36  2003  left_only
37  2004  left_only
38  2005  left_only
39  2006  left_only
40  2007  left_only
41  2008  left_only
42  2009  left_only
43  2010  left_only
44  2011       both
45  2012       both
46  2013       both
47  2014       both
48  2015  left_only


In [55]:
print(filtered_Year_df.head())
merge_counts = filtered_Year_df['_merge'].value_counts()
print(merge_counts)


   Year     _merge
0  1967  left_only
1  1968  left_only
2  1969  left_only
3  1970  left_only
4  1971  left_only
left_only     47
both           4
right_only     0
Name: _merge, dtype: int64


In [56]:
newfiltered_Year_df = filtered_Year_df[[ 'Year', '_merge']].query('_merge == "left_only"')

print(newfiltered_Year_df.head())
merge_counts1 = newfiltered_Year_df['_merge'].value_counts()
print(merge_counts1)

   Year     _merge
0  1967  left_only
1  1968  left_only
2  1969  left_only
3  1970  left_only
4  1971  left_only
left_only     47
right_only     0
both           0
Name: _merge, dtype: int64


In [57]:
YearDim = pd.concat([unique_Year_df, newfiltered_Year_df], ignore_index=True)

print(YearDim.head())


   Year     _merge
0  2014        NaN
1  2012        NaN
2  2013        NaN
3  2011        NaN
4  1967  left_only


In [58]:
YearDim.reset_index(drop=False, inplace=True)
YearDim.rename(columns={'index': 'Year_id'}, inplace=True)
YearDim['Year_id'] += 1  # Start the index at 1
YearDim.head()


,Year_id,Year,_merge
0,1,2014,NaN
1,2,2012,NaN
2,3,2013,NaN
3,4,2011,NaN
4,5,1967,left_only


In [59]:
YearDim.to_csv('/Users/kadam/Desktop/CS689/Project/Data/YearDim.csv', index=False)  

print(f'DataFrame has been successfully written')

DataFrame has been successfully written


### Month Dimension

In [60]:
month_df = salesdata_df[['Order Month', 'Order Year']]

In [61]:
month_df = month_df.rename(columns={
    'Order Year': 'Year',
    'Order Month': 'Month'
})

print(month_df)

       Month  Year
0         10  2014
1         10  2012
2         10  2012
3         10  2012
4         10  2012
...      ...   ...
51285      9  2012
51286      9  2012
51287      9  2012
51288      9  2012
51289      8  2011

[51290 rows x 2 columns]


In [62]:
duplicate_counts1 = month_df.groupby(['Month','Year']).size().reset_index(name='Count')
duplicates1 = duplicate_counts1[duplicate_counts1['Count'] > 1] ##To find duplicate values
print(duplicates1)

    Month  Year  Count
0       1  2011    433
1       1  2012    539
2       1  2013    709
3       1  2014    918
4       2  2011    378
5       2  2012    429
6       2  2013    605
7       2  2014    756
8       3  2011    539
9       3  2012    662
10      3  2013    766
11      3  2014   1068
12      4  2011    564
13      4  2012    657
14      4  2013    785
15      4  2014   1051
16      5  2011    566
17      5  2012    840
18      5  2013   1057
19      5  2014   1284
20      6  2011    917
21      6  2012   1136
22      6  2013   1530
23      6  2014   1748
24      7  2011    495
25      7  2012    659
26      7  2013    925
27      7  2014   1087
28      8  2011    878
29      8  2012   1083
30      8  2013   1442
31      8  2014   1675
32      9  2011   1052
33      9  2012   1223
34      9  2013   1682
35      9  2014   2018
36     10  2011    774
37     10  2012    990
38     10  2013   1100
39     10  2014   1626
40     11  2011   1139
41     11  2012   1460
42     11  

In [63]:
newmonth_df = month_df.drop_duplicates(subset=['Month','Year'])

newmonth_df.head()

,Month,Year
0,10,2014
1,10,2012
6,9,2013
11,3,2013
13,9,2014


In [64]:
# Adding a Quarter column based on the Month
newmonth1_df = newmonth_df.copy()

newmonth1_df.loc[:, 'Quarter'] = pd.to_datetime(newmonth1_df['Month'], format='%m').dt.quarter

newmonth1_df.head()


,Month,Year,Quarter
0,10,2014,4
1,10,2012,4
6,9,2013,3
11,3,2013,1
13,9,2014,3


In [65]:
MonthDim = newmonth1_df.copy()

MonthDim.reset_index(drop=False, inplace=True)
MonthDim.rename(columns={'index': 'MonthId'}, inplace=True)
MonthDim['MonthId'] += 1  # Start the index at 1
MonthDim.head()

,MonthId,Month,Year,Quarter
0,1,10,2014,4
1,2,10,2012,4
2,7,9,2013,3
3,12,3,2013,1
4,14,9,2014,3


In [66]:
MonthDim.to_csv('/Users/kadam/Desktop/CS689/Project/Data/MonthDim.csv', index=False)  

print(f'DataFrame has been successfully written')

DataFrame has been successfully written


### Population Income Dimension

In [67]:
PopIncome_df = incomedata_df[['Income_Category', 'Region_ID','Country', 'ISO']]

In [68]:
duplicate_counts = PopIncome_df.groupby(['Income_Category', 'Region_ID','Country', 'ISO']).size().reset_index(name='Count')
duplicates = duplicate_counts[duplicate_counts['Count'] > 1] ##To find duplicate values
print(duplicates)

     Income_Category  Region_ID                   Country  ISO  Count
0                 d1          1               Puerto Rico  pri     49
1                 d1          1  United States of America  usa     49
2                 d1          2                   Burundi  bdi     49
3                 d1          2                   Comoros  com     49
4                 d1          2                  Djibouti  dji     49
...              ...        ...                       ...  ...    ...
1893              d9         29                     Tonga  ton     49
1896              d9         29                  Viet Nam  vnm     49
1897              d9         30                    Taiwan  twn     49
1898              d9         31                 Argentina  arg     49
1899              d9         32                  Colombia  col     49

[1770 rows x 5 columns]


In [69]:
## dropping the duplicate values
NewPopIncome_df = PopIncome_df.drop_duplicates(subset=['Income_Category', 'Region_ID','Country', 'ISO'])

print(NewPopIncome_df)

      Income_Category  Region_ID      Country  ISO
0                  d1         27  Afghanistan  afg
1                  d2         27  Afghanistan  afg
2                  d3         27  Afghanistan  afg
3                  d4         27  Afghanistan  afg
4                  d5         27  Afghanistan  afg
...               ...        ...          ...  ...
84735              d6          4     Zimbabwe  zwe
84736              d7          4     Zimbabwe  zwe
84737              d8          4     Zimbabwe  zwe
84738              d9          4     Zimbabwe  zwe
84739             d10          4     Zimbabwe  zwe

[1900 rows x 4 columns]


In [70]:
# Making the dimension into SCD2
from datetime import datetime

def apply_scd2(NewPopIncome_df):
    NewPopIncome_df['Effective_From'] = pd.to_datetime(datetime.now().date())  # Setting the current date as the effective date for new records
    NewPopIncome_df['Effective_To'] = pd.to_datetime('2262-04-11')  # Set a future date for the current records

    NewPopIncome_df['CurrentFlag'] = NewPopIncome_df.duplicated(subset=['Income_Category', 'Region_ID'], keep='last')

    # Update the existing records
    NewPopIncome_df.loc[NewPopIncome_df['CurrentFlag'], 'Effective_To'] = pd.to_datetime(datetime.now().date())

    return NewPopIncome_df

# Applying SCD2
NewPopIncome_scd2 = apply_scd2(NewPopIncome_df)

print(NewPopIncome_scd2)

      Income_Category  Region_ID      Country  ISO Effective_From  \
0                  d1         27  Afghanistan  afg     2023-12-10   
1                  d2         27  Afghanistan  afg     2023-12-10   
2                  d3         27  Afghanistan  afg     2023-12-10   
3                  d4         27  Afghanistan  afg     2023-12-10   
4                  d5         27  Afghanistan  afg     2023-12-10   
...               ...        ...          ...  ...            ...   
84735              d6          4     Zimbabwe  zwe     2023-12-10   
84736              d7          4     Zimbabwe  zwe     2023-12-10   
84737              d8          4     Zimbabwe  zwe     2023-12-10   
84738              d9          4     Zimbabwe  zwe     2023-12-10   
84739             d10          4     Zimbabwe  zwe     2023-12-10   

      Effective_To  CurrentFlag  
0       2023-12-10         True  
1       2023-12-10         True  
2       2023-12-10         True  
3       2023-12-10         True  
4

/var/folders/6n/g0nqd9qx36g6tzn8__zgtbp80000gn/T/ipykernel_39797/4123980507.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NewPopIncome_df['Effective_From'] = pd.to_datetime(datetime.now().date())  # Setting the current date as the effective date for new records
/var/folders/6n/g0nqd9qx36g6tzn8__zgtbp80000gn/T/ipykernel_39797/4123980507.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NewPopIncome_df['Effective_To'] = pd.to_datetime('2262-04-11')  # Set a future date for the current records
/var/fol

In [71]:
PopIncomeDim = NewPopIncome_scd2.copy()

PopIncomeDim.reset_index(drop=False, inplace=True)
PopIncomeDim.rename(columns={'index': 'PopIncome_Id'}, inplace=True)
PopIncomeDim['PopIncome_Id'] += 1  # Start the index at 1
PopIncomeDim.head()

,PopIncome_Id,Income_Category,Region_ID,Country,ISO,Effective_From,Effective_To,CurrentFlag
0,1,d1,27,Afghanistan,afg,2023-12-10,2023-12-10,True
1,2,d2,27,Afghanistan,afg,2023-12-10,2023-12-10,True
2,3,d3,27,Afghanistan,afg,2023-12-10,2023-12-10,True
3,4,d4,27,Afghanistan,afg,2023-12-10,2023-12-10,True
4,5,d5,27,Afghanistan,afg,2023-12-10,2023-12-10,True


In [72]:
PopIncomeDim = PopIncomeDim.rename(columns={
    'Region_ID': 'PopRegion_number',
    'Country': 'PopCountry'
})

print(PopIncomeDim)

      PopIncome_Id Income_Category  PopRegion_number   PopCountry  ISO  \
0                1              d1                27  Afghanistan  afg   
1                2              d2                27  Afghanistan  afg   
2                3              d3                27  Afghanistan  afg   
3                4              d4                27  Afghanistan  afg   
4                5              d5                27  Afghanistan  afg   
...            ...             ...               ...          ...  ...   
1895         84736              d6                 4     Zimbabwe  zwe   
1896         84737              d7                 4     Zimbabwe  zwe   
1897         84738              d8                 4     Zimbabwe  zwe   
1898         84739              d9                 4     Zimbabwe  zwe   
1899         84740             d10                 4     Zimbabwe  zwe   

     Effective_From Effective_To  CurrentFlag  
0        2023-12-10   2023-12-10         True  
1        2023-1

In [73]:
PopIncomeDim.to_csv('/Users/kadam/Desktop/CS689/Project/Data/PopIncomeDim.csv', index=False)  

print(f'DataFrame has been successfully written')

DataFrame has been successfully written


### Fact Tables 

### SalesFact Table

In [267]:
salesdata_df1 = salesdata_df[['Sales', 'Quantity', 'Discount', 'Profit']]

In [268]:
sales_fact_data = pd.concat([salesdata_df1, ProductDim, CustomerDim, DateDim, LocationDim, MarketDim, ShipmentDim, OrderDim, PopIncomeDim], axis=1)

In [269]:
print(sales_fact_data)

        Sales  Quantity  Discount  Profit  ProdDim_Id       Product_id  \
0       13.08         3       0.0    4.56         1.0  OFF-LA-10002782   
1      252.16         8       0.0   90.72         2.0  FUR-FU-10004015   
2      193.28         2       0.0   54.08         3.0  FUR-BO-10002352   
3       35.44         4       0.0    4.96         4.0  OFF-BI-10004428   
4       71.60         2       0.0   11.44         5.0  OFF-AR-10004594   
...       ...       ...       ...     ...         ...              ...   
51285   11.07         1       0.0    3.42         NaN              NaN   
51286   61.44         2       0.0   18.42         NaN              NaN   
51287   80.52         4       0.0   20.88         NaN              NaN   
51288  130.44         4       0.0   33.84         NaN              NaN   
51289   31.62         1       0.0   12.96         NaN              NaN   

                             Product_name Product_category  \
0      Hon File Folder Labels, Adjustable  Office

In [270]:
# calculating unit price
sales_fact_data['Unit_Price'] = sales_fact_data['Sales'] / sales_fact_data['Quantity']

In [271]:
SalesFact = sales_fact_data[['Sales', 'Quantity', 'Discount', 'Profit','Unit_Price','ProdDim_Id','customerId', 'shipment_id', 'Market_Id', 'OrderId','Location_Id', 'DateDim_Id', 'PopIncome_Id']]

In [272]:
print(SalesFact)

        Sales  Quantity  Discount  Profit  Unit_Price  ProdDim_Id  customerId  \
0       13.08         3       0.0    4.56        4.36         1.0         1.0   
1      252.16         8       0.0   90.72       31.52         2.0         2.0   
2      193.28         2       0.0   54.08       96.64         3.0         7.0   
3       35.44         4       0.0    4.96        8.86         4.0        12.0   
4       71.60         2       0.0   11.44       35.80         5.0        14.0   
...       ...       ...       ...     ...         ...         ...         ...   
51285   11.07         1       0.0    3.42       11.07         NaN         NaN   
51286   61.44         2       0.0   18.42       30.72         NaN         NaN   
51287   80.52         4       0.0   20.88       20.13         NaN         NaN   
51288  130.44         4       0.0   33.84       32.61         NaN         NaN   
51289   31.62         1       0.0   12.96       31.62         NaN         NaN   

       shipment_id  Market_

In [273]:
print(SalesFact.dtypes)

Sales           float64
Quantity          int64
Discount        float64
Profit          float64
Unit_Price      float64
ProdDim_Id      float64
customerId      float64
shipment_id       int64
Market_Id       float64
OrderId         float64
Location_Id     float64
DateDim_Id      float64
PopIncome_Id    float64
dtype: object


In [274]:
SalesFact = SalesFact.dropna(subset=['Location_Id'])

# Now, you can convert the 'Location_Id' column to int64
#SalesFact['Location_Id'] = SalesFact['Location_Id'].astype('int64')

In [275]:
SalesFact['customerId'] = SalesFact['customerId'].astype('Int64')
SalesFact['Market_Id'] = SalesFact['Market_Id'].astype('Int64')
SalesFact['ProdDim_Id'] = SalesFact['ProdDim_Id'].astype('Int64')
SalesFact['PopIncome_Id'] = SalesFact['PopIncome_Id'].astype('Int64')

In [276]:
SalesFact = SalesFact.dropna(subset=['Market_Id'])

In [277]:
SalesFact.to_csv('/Users/kadam/Desktop/CS689/Project/Data/SalesFact.csv', index=False)  

print(f'DataFrame has been successfully written')

DataFrame has been successfully written


### IncomeFact Table

In [193]:
incomedata_df1 = incomedata_df[['GDP_PPC_USD2011', 'Population', 'Gini_coeff', 'Net_Income']]

In [194]:
income_fact_data = pd.concat([incomedata_df1, YearDim, PopIncomeDim], axis=1)

In [195]:
column_row_counts = income_fact_data.count()
print(column_row_counts)

GDP_PPC_USD2011     85220
Population          85220
Gini_coeff          84880
Net_Income          85220
Year_id                51
Year                   51
_merge                 47
PopIncome_Id         1900
Income_Category      1900
PopRegion_number     1900
PopCountry           1900
ISO                  1900
Effective_From       1900
Effective_To         1900
CurrentFlag          1900
dtype: int64


In [196]:
IncomeFact = income_fact_data[['PopIncome_Id', 'Year_id', 'GDP_PPC_USD2011', 'Population', 'Gini_coeff', 'Net_Income']]

In [197]:
print(IncomeFact.dtypes)

PopIncome_Id       float64
Year_id            float64
GDP_PPC_USD2011    float64
Population         float64
Gini_coeff         float64
Net_Income         float64
dtype: object


In [198]:
# Replace NaN values with 0 and convert to int
IncomeFact['PopIncome_Id'] = IncomeFact['PopIncome_Id'].fillna(0).astype('int64')
IncomeFact = IncomeFact.dropna(subset=['Year_id'])

IncomeFact['Population'] = IncomeFact['Population'].fillna(0).astype('int64')


/var/folders/6n/g0nqd9qx36g6tzn8__zgtbp80000gn/T/ipykernel_39797/3068759987.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IncomeFact['PopIncome_Id'] = IncomeFact['PopIncome_Id'].fillna(0).astype('int64')


In [199]:
IncomeFact.to_csv('/Users/kadam/Desktop/CS689/Project/Data/IncomeFact.csv', index=False)  

print(f'DataFrame has been successfully written')

DataFrame has been successfully written


### SalesCumFact Table

In [279]:
Salescumfact_data = pd.concat([salesdata_df1,ProductDim, CustomerDim, MonthDim, LocationDim, MarketDim, ShipmentDim, OrderDim, PopIncomeDim], axis=1)

In [280]:
print(Salescumfact_data)

        Sales  Quantity  Discount  Profit  ProdDim_Id       Product_id  \
0       13.08         3       0.0    4.56         1.0  OFF-LA-10002782   
1      252.16         8       0.0   90.72         2.0  FUR-FU-10004015   
2      193.28         2       0.0   54.08         3.0  FUR-BO-10002352   
3       35.44         4       0.0    4.96         4.0  OFF-BI-10004428   
4       71.60         2       0.0   11.44         5.0  OFF-AR-10004594   
...       ...       ...       ...     ...         ...              ...   
51285   11.07         1       0.0    3.42         NaN              NaN   
51286   61.44         2       0.0   18.42         NaN              NaN   
51287   80.52         4       0.0   20.88         NaN              NaN   
51288  130.44         4       0.0   33.84         NaN              NaN   
51289   31.62         1       0.0   12.96         NaN              NaN   

                             Product_name Product_category  \
0      Hon File Folder Labels, Adjustable  Office

In [282]:
 #Group by month and calculate averages
Salescumfact_data['order_date'] = pd.to_datetime(Salescumfact_data['order_date'])
    
monthly_averages = Salescumfact_data.groupby(Salescumfact_data['order_date'].dt.to_period("M")).agg({
    'Sales': 'mean',
    'Profit': 'mean',
    'Quantity': 'mean',
    'Discount': 'mean'
}).reset_index()

# Merge the averages back into the original DataFrame
Salescumfact_data = pd.merge(Salescumfact_data, monthly_averages, left_on=Salescumfact_data['order_date'].dt.to_period("M"), right_on=monthly_averages['order_date'], suffixes=('', '_avg'))

# Print or use the resulting DataFrame
print(Salescumfact_data)

         key_0      Sales  Quantity  Discount     Profit  ProdDim_Id  \
0      2014-10    13.0800         3      0.00     4.5600         1.0   
1      2014-10    56.1200         2      0.00    21.3200         6.0   
2      2014-10   341.5200         2      0.00    13.6400        11.0   
3      2014-10   308.2800         7      0.40  -149.1000        19.0   
4      2014-10   928.2560         2      0.20   -81.2240        74.0   
...        ...        ...       ...       ...        ...         ...   
25748  2012-01    20.5800         2      0.00     8.2200         NaN   
25749  2012-01   118.3500         3      0.00    36.6300         NaN   
25750  2012-01   235.8528         2      0.17   -25.6272         NaN   
25751  2012-01   128.6280         3      0.10    19.9980         NaN   
25752  2012-01  2117.2200        14      0.50 -1736.2800         NaN   

            Product_id                                   Product_name  \
0      OFF-LA-10002782             Hon File Folder Labels, Adj

In [283]:
SalesCumFact = Salescumfact_data[['order_date_avg', 'Sales_avg', 'Profit_avg', 'Quantity_avg', 'Discount_avg', 'ProdDim_Id','customerId', 'shipment_id', 'Market_Id', 'OrderId','Location_Id', 'MonthId', 'PopIncome_Id' ]]

In [284]:
print(SalesCumFact)

      order_date_avg   Sales_avg  Profit_avg  Quantity_avg  Discount_avg  \
0            2014-10  266.400124   30.649988      3.916049      0.122469   
1            2014-10  266.400124   30.649988      3.916049      0.122469   
2            2014-10  266.400124   30.649988      3.916049      0.122469   
3            2014-10  266.400124   30.649988      3.916049      0.122469   
4            2014-10  266.400124   30.649988      3.916049      0.122469   
...              ...         ...         ...           ...           ...   
25748        2012-01  242.373597   12.148751      3.461538      0.148731   
25749        2012-01  242.373597   12.148751      3.461538      0.148731   
25750        2012-01  242.373597   12.148751      3.461538      0.148731   
25751        2012-01  242.373597   12.148751      3.461538      0.148731   
25752        2012-01  242.373597   12.148751      3.461538      0.148731   

       ProdDim_Id  customerId  shipment_id  Market_Id  OrderId  Location_Id  \
0       

In [285]:
SalesCumFact = SalesCumFact.dropna(subset=['Market_Id'])
SalesCumFact = SalesCumFact.dropna(subset=['Location_Id'])
SalesCumFact = SalesCumFact.dropna(subset=['OrderId'])
SalesCumFact = SalesCumFact.dropna(subset=['shipment_id'])
SalesCumFact = SalesCumFact.dropna(subset=['customerId'])
SalesCumFact = SalesCumFact.dropna(subset=['PopIncome_Id'])

In [286]:
print(SalesCumFact.dtypes)

order_date_avg    period[M]
Sales_avg           float64
Profit_avg          float64
Quantity_avg        float64
Discount_avg        float64
ProdDim_Id          float64
customerId          float64
shipment_id           int64
Market_Id           float64
OrderId             float64
Location_Id         float64
MonthId             float64
PopIncome_Id        float64
dtype: object


In [287]:

SalesCumFact = SalesCumFact.dropna(subset=['ProdDim_Id'])
SalesCumFact = SalesCumFact.dropna(subset=['MonthId'])

In [288]:
SalesCumFact['customerId'] = SalesCumFact['customerId'].astype(int)
SalesCumFact['Market_Id'] = SalesCumFact['Market_Id'].astype(int)
SalesCumFact['OrderId'] = SalesCumFact['OrderId'].astype(int)
SalesCumFact['Location_Id'] = SalesCumFact['Location_Id'].astype(int)
SalesCumFact['ProdDim_Id'] = SalesCumFact['ProdDim_Id'].astype(int)
SalesCumFact['MonthId'] = SalesCumFact['MonthId'].astype(int)
SalesCumFact['PopIncome_Id'] = SalesCumFact['PopIncome_Id'].astype(int)

In [289]:
SalesCumFact.to_csv('/Users/kadam/Desktop/CS689/Project/Data/SalesCumFact.csv', index=False)  

print(f'DataFrame has been successfully written')

DataFrame has been successfully written


### ShipmentFact Table

In [249]:
shipmentfact_data = salesdata_df[['Ship Date', 'Shipping Cost']]

In [250]:
Shipmentfact_data = pd.concat([shipmentfact_data, ProductDim, CustomerDim, DateDim, LocationDim, ShipmentDim, OrderDim], axis=1)

In [257]:
ShipmentFact = Shipmentfact_data[['Shipping Cost', 'shipment_date', 'ProdDim_Id', 'customerId', 'DateDim_Id', 'Location_Id', 'shipment_id', 'OrderId']]

In [258]:
ShipmentFact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51290 entries, 0 to 51289
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Shipping Cost  51290 non-null  float64
 1   shipment_date  51290 non-null  object 
 2   ProdDim_Id     10768 non-null  float64
 3   customerId     24959 non-null  float64
 4   DateDim_Id     1430 non-null   float64
 5   Location_Id    3819 non-null   float64
 6   shipment_id    51290 non-null  int64  
 7   OrderId        25753 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 3.5+ MB


In [259]:
columns_to_convert = ['ProdDim_Id', 'customerId', 'DateDim_Id', 'Location_Id', 'OrderId']

ShipmentFact[columns_to_convert] = ShipmentFact[columns_to_convert].astype('Int64', errors='ignore') 


print(ShipmentFact)

       Shipping Cost shipment_date  ProdDim_Id  customerId  DateDim_Id  \
0               1.03    2014-10-06           1           1           1   
1              13.45    2012-10-20           2           2           2   
2               9.63    2012-10-20           3           7           7   
3               1.37    2012-10-20           4          12          12   
4               3.79    2012-10-20           5          14          14   
...              ...           ...         ...         ...         ...   
51285           1.98    2012-09-07        <NA>        <NA>        <NA>   
51286          13.02    2012-09-07        <NA>        <NA>        <NA>   
51287           8.78    2012-09-07        <NA>        <NA>        <NA>   
51288          18.97    2012-09-07        <NA>        <NA>        <NA>   
51289           2.40    2011-08-29        <NA>        <NA>        <NA>   

       Location_Id  shipment_id  OrderId  
0                1            1        1  
1                2       

/var/folders/6n/g0nqd9qx36g6tzn8__zgtbp80000gn/T/ipykernel_39797/640022433.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ShipmentFact[columns_to_convert] = ShipmentFact[columns_to_convert].astype('Int64', errors='ignore')


In [260]:
columns_to_dropna = ['ProdDim_Id', 'customerId', 'DateDim_Id', 'Location_Id', 'OrderId']
ShipmentFact = ShipmentFact.dropna(subset=columns_to_dropna)

In [263]:
ShipmentFact = ShipmentFact.rename(columns={
    'Shipping Cost': 'Shipping_Cost',
})

print(ShipmentFact)

      Shipping_Cost shipment_date  ProdDim_Id  customerId  DateDim_Id  \
0              1.03    2014-10-06           1           1           1   
1             13.45    2012-10-20           2           2           2   
2              9.63    2012-10-20           3           7           7   
3              1.37    2012-10-20           4          12          12   
4              3.79    2012-10-20           5          14          14   
...             ...           ...         ...         ...         ...   
1425          28.09    2013-08-29        2742        2906       47444   
1426          27.26    2013-08-29        2749        2908       48576   
1427          12.40    2011-12-11        2755        2910       49550   
1428          74.78    2014-06-17        2756        2912       49729   
1429           1.56    2013-09-11        2760        2914       49895   

      Location_Id  shipment_id  OrderId  
0               1            1        1  
1               2            2        2

In [264]:
ShipmentFact.to_csv('/Users/kadam/Desktop/CS689/Project/Data/ShipmentFact.csv', index=False)  

print(f'DataFrame has been successfully written')

DataFrame has been successfully written


### Populating dataframe into PostgreSQL 

In [126]:
import psycopg2
import pandas as pd


db_params = {
    "user": "cs689",
    "password": "14nautanki",
    "host": "localhost",
    "port": "5432",
    "database": "Project"
}

connection = psycopg2.connect(**db_params)

cursor = connection.cursor()

df = pd.read_csv('/Users/kadam/Desktop/CS689/Project/Data/ProductDim.csv')  

# Iterate through the DataFrame and insert rows into the Dim_Ship table
for index, row in df.iterrows():
    sql = "INSERT INTO ProductDim (ProdDim_Id, Product_id, Product_name, Product_category, Product_subcategory, Effective_From, Effective_To, CurrentFlag ) VALUES (%s, %s, %s, %s, %s,  %s, %s, %s)"
    values = (row['ProdDim_Id'], row['Product_id'], row['Product_name'], row['Product_category'], row['Product_subcategory'], row['Effective_From'], row['Effective_To'], row['CurrentFlag'])
    cursor.execute(sql, values)


connection.commit()

cursor.close()
connection.close()


In [127]:

db_params = {
    "user": "cs689",
    "password": "14nautanki",
    "host": "localhost",
    "port": "5432",
    "database": "Project"
}

connection = psycopg2.connect(**db_params)

cursor = connection.cursor()

df = pd.read_csv('/Users/kadam/Desktop/CS689/Project/Data/ShipmentDim.csv')  

# Iterate through the DataFrame and insert rows into the Dim_Ship table
for index, row in df.iterrows():
    sql = "INSERT INTO ShipmentDim (shipment_id, shipment_date, shipment_mode, Effective_From, Effective_To, CurrentFlag ) VALUES (%s, %s, %s, %s, %s, %s)"
    values = (row['shipment_id'], row['shipment_date'], row['shipment_mode'], row['Effective_From'], row['Effective_To'], row['CurrentFlag'])
    cursor.execute(sql, values)


connection.commit()

cursor.close()
connection.close()


In [135]:

db_params = {
    "user": "cs689",
    "password": "14nautanki",
    "host": "localhost",
    "port": "5432",
    "database": "Project"
}

connection = psycopg2.connect(**db_params)

cursor = connection.cursor()

df = pd.read_csv('/Users/kadam/Desktop/CS689/Project/Data/OrderDim.csv')  

# Iterate through the DataFrame and insert rows into the Dim_Ship table
for index, row in df.iterrows():
    sql = "INSERT INTO OrderDim (OrderId, order_id, order_date, order_priority) VALUES (%s, %s, %s, %s)"
    values = (row['OrderId'], row['order_id'], row['order_date'], row['order_priority'])
    cursor.execute(sql, values)


connection.commit()

cursor.close()
connection.close()

In [136]:

db_params = {
    "user": "cs689",
    "password": "14nautanki",
    "host": "localhost",
    "port": "5432",
    "database": "Project"
}

connection = psycopg2.connect(**db_params)

cursor = connection.cursor()

df = pd.read_csv('/Users/kadam/Desktop/CS689/Project/Data/MarketDim.csv')  


for index, row in df.iterrows():
    sql = "INSERT INTO MarketDim (Market_Id, market_segment, market_category, market_region) VALUES (%s, %s, %s, %s)"
    values = (row['Market_Id'], row['market_segment'], row['market_category'], row['market_region'])
    cursor.execute(sql, values)


connection.commit()

cursor.close()
connection.close()

In [145]:
db_params = {
    "user": "cs689",
    "password": "14nautanki",
    "host": "localhost",
    "port": "5432",
    "database": "Project"
}

connection = psycopg2.connect(**db_params)

cursor = connection.cursor()

df = pd.read_csv('/Users/kadam/Desktop/CS689/Project/Data/CustomerDim.csv')  

# Iterate through the DataFrame and insert rows into the Dim_Ship table
for index, row in df.iterrows():
    sql = "INSERT INTO CustomerDim (customerId, customer_id, customer_name, customer_city, customer_state, customer_country, Effective_From, Effective_To, CurrentFlag ) VALUES (%s, %s, %s, %s, %s, %s,  %s, %s, %s)"
    values = (row['customerId'], row['customer_id'], row['customer_name'], row['customer_city'], row['customer_state'], row['customer_country'], row['Effective_From'], row['Effective_To'], row['CurrentFlag'])
    cursor.execute(sql, values)


connection.commit()

cursor.close()
connection.close()

In [146]:
db_params = {
    "user": "cs689",
    "password": "14nautanki",
    "host": "localhost",
    "port": "5432",
    "database": "Project"
}

connection = psycopg2.connect(**db_params)

cursor = connection.cursor()

df = pd.read_csv('/Users/kadam/Desktop/CS689/Project/Data/LocationDim.csv')  


for index, row in df.iterrows():
    sql = "INSERT INTO LocationDim (Location_Id, Region, City, State, Country) VALUES (%s, %s, %s, %s, %s)"
    values = (row['Location_Id'], row['Region'], row['City'], row['State'], row['Country'])
    cursor.execute(sql, values)


connection.commit()

cursor.close()
connection.close()

In [147]:
db_params = {
    "user": "cs689",
    "password": "14nautanki",
    "host": "localhost",
    "port": "5432",
    "database": "Project"
}

connection = psycopg2.connect(**db_params)

cursor = connection.cursor()

df = pd.read_csv('/Users/kadam/Desktop/CS689/Project/Data/DateDim.csv')  


for index, row in df.iterrows():
    sql = "INSERT INTO DateDim (DateDim_Id, Date, Year, Month, Day_of_Week, Week_Number) VALUES (%s, %s, %s, %s, %s, %s)"
    values = (row['DateDim_Id'], row['Date'], row['Year'], row['Month'], row['Day_of_Week'], row['Week_Number'])
    cursor.execute(sql, values)


connection.commit()

cursor.close()
connection.close()

In [148]:
db_params = {
    "user": "cs689",
    "password": "14nautanki",
    "host": "localhost",
    "port": "5432",
    "database": "Project"
}

connection = psycopg2.connect(**db_params)

cursor = connection.cursor()

df = pd.read_csv('/Users/kadam/Desktop/CS689/Project/Data/YearDim.csv')  

# Iterate through the DataFrame and insert rows into the Dim_Ship table
for index, row in df.iterrows():
    sql = "INSERT INTO YearDim (Year_id, Year) VALUES (%s, %s)"
    values = (row['Year_id'], row['Year'])
    cursor.execute(sql, values)


connection.commit()

cursor.close()
connection.close()

In [150]:

db_params = {
    "user": "cs689",
    "password": "14nautanki",
    "host": "localhost",
    "port": "5432",
    "database": "Project"
}

connection = psycopg2.connect(**db_params)

cursor = connection.cursor()

df = pd.read_csv('/Users/kadam/Desktop/CS689/Project/Data/MonthDim.csv')  

for index, row in df.iterrows():
    # Convert numpy.int64 to int
    values = (
        int(row['MonthId']),
        int(row['Month']),
        int(row['Year']),
        int(row['Quarter'])
    )
    sql = "INSERT INTO MonthDim (MonthId, Month, Year, Quarter) VALUES (%s, %s, %s, %s)"
    cursor.execute(sql, values)


connection.commit()

cursor.close()
connection.close()

In [151]:
db_params = {
    "user": "cs689",
    "password": "14nautanki",
    "host": "localhost",
    "port": "5432",
    "database": "Project"
}

connection = psycopg2.connect(**db_params)

cursor = connection.cursor()

df = pd.read_csv('/Users/kadam/Desktop/CS689/Project/Data/PopIncomeDim.csv')  


for index, row in df.iterrows():
    sql = "INSERT INTO PopIncomeDim (PopIncome_Id, Income_Category, PopRegion_number, PopCountry, ISO, Effective_From, Effective_To, CurrentFlag) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    values = (row['PopIncome_Id'], row['Income_Category'], row['PopRegion_number'], row['PopCountry'], row['ISO'], row['Effective_From'], row['Effective_To'], row['CurrentFlag'])
    cursor.execute(sql, values)


connection.commit()

cursor.close()
connection.close()

### Populating Fact tables

In [278]:
db_params = {
    "user": "cs689",
    "password": "14nautanki",
    "host": "localhost",
    "port": "5432",
    "database": "Project"
}

connection = psycopg2.connect(**db_params)

cursor = connection.cursor()

df = pd.read_csv('/Users/kadam/Desktop/CS689/Project/Data/SalesFact.csv')  


for index, row in df.iterrows():
    sql = "INSERT INTO SalesFact (Sales, Quantity, Discount, Profit, Unit_Price, ProdDim_Id, customerId, shipment_id, Market_Id, OrderId, DateDim_Id, PopIncome_Id) VALUES ( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    values = (row['Sales'], row['Quantity'], row['Discount'], row['Profit'], row['Unit_Price'], row['ProdDim_Id'], row['customerId'], row['shipment_id'], row['Market_Id'], row['OrderId'], row['DateDim_Id'], row['PopIncome_Id'])
    cursor.execute(sql, values)


connection.commit()

cursor.close()
connection.close()

In [200]:
db_params = {
    "user": "cs689",
    "password": "14nautanki",
    "host": "localhost",
    "port": "5432",
    "database": "Project"
}

connection = psycopg2.connect(**db_params)

cursor = connection.cursor()

df = pd.read_csv('/Users/kadam/Desktop/CS689/Project/Data/IncomeFact.csv')  


for index, row in df.iterrows():
    sql = "INSERT INTO IncomeFact (PopIncome_Id, Year_id, GDP_PPC_USD2011, Population, Gini_coeff, Net_Income) VALUES (%s, %s, %s, %s, %s, %s)"
    values = (row['PopIncome_Id'], row['Year_id'], row['GDP_PPC_USD2011'], row['Population'], row['Gini_coeff'], row['Net_Income'])
    cursor.execute(sql, values)


connection.commit()

cursor.close()
connection.close()

In [290]:
db_params = {
    "user": "cs689",
    "password": "14nautanki",
    "host": "localhost",
    "port": "5432",
    "database": "Project"
}

connection = psycopg2.connect(**db_params)

cursor = connection.cursor()

df = pd.read_csv('/Users/kadam/Desktop/CS689/Project/Data/SalesCumFact.csv')  


for index, row in df.iterrows():
    sql = "INSERT INTO SalesCumFact (Sales_avg, Profit_avg, Quantity_avg, Discount_avg, ProdDim_Id, customerId, shipment_id, Market_Id, OrderId, MonthId, PopIncome_Id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    values = (row['Sales_avg'], row['Profit_avg'], row['Quantity_avg'], row['Discount_avg'], row['ProdDim_Id'], row['customerId'], row['shipment_id'], row['Market_Id'], row['OrderId'], row['MonthId'], row['PopIncome_Id'])
    cursor.execute(sql, values)


connection.commit()

cursor.close()
connection.close()

In [266]:
db_params = {
    "user": "cs689",
    "password": "14nautanki",
    "host": "localhost",
    "port": "5432",
    "database": "Project"
}

connection = psycopg2.connect(**db_params)

cursor = connection.cursor()

df = pd.read_csv('/Users/kadam/Desktop/CS689/Project/Data/ShipmentFact.csv')  


for index, row in df.iterrows():
    sql = "INSERT INTO ShipmentFact (Shipping_Cost, shipment_date, ProdDim_Id, customerId, shipment_id, OrderId, Location_Id, DateDim_Id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    values = (row['Shipping_Cost'], row['shipment_date'], row['ProdDim_Id'], row['customerId'], row['shipment_id'], row['OrderId'],row['Location_Id'], row['DateDim_Id'])
    cursor.execute(sql, values)


connection.commit()

cursor.close()
connection.close()